# Introduction to constrained multi-objective management optimization (under uncertainty)

### yeah, its getting deep!

In the two PESTPP-OPT notebooks, we introduced the concept of constrained management optimization under uncertainty.  We saw standard risk-neutral optimization and then piled on the learning and concepts with the idea of chances, chance constraints, risk/reliability, and stacks.  So if you are reading this notebook...

Ok, so now let's talk about the nature of constraints and objective functions.  In the Freyberg example, we have been treating the sw-gw exchange flux and the aggregate groundwater extraction rate for each stress period as a "hard" inequality constraint: thou shall not violate! But in many settings there is a general stakeholder preference to avoid unwanted outcomes but the exact nature of that avoidance is not known: "Sure we want to keep some groundwater flowing into the surface-water system but we also want have plenty of water to drink".  Very imprecise...so how can we deal with this is science nerds?

Well, one way is to use so-called "multi-objective" optimization, where the goal is to map the trade off between competing objectives. Unfortunately, this kind of trade-off mapping is very (very) computationally expensive because in most cases, we have to resort to "global" evolutionary-type algorithms.  Note that "multi-objective" doesnt mean go crazy with objectives.  Five or six is probably the most that be used for algorithmic reasons.  

If you are intersted in learning more about multi-objective optimization, #LMGTFY: "pareto frontier", "pareto dominance", "nsga-II", etc...


### Admin

Start off with the usual loading of dependencies and preparing model and PEST files. We will be continuing to work with the modified-Freyberg model (see "intro to model" notebook), and the high-dimensional PEST dataset prepared in the "pstfrom pest setup" and "obs and weights" notebooks. 

For the purposes of this notebook, you do not require familiarity with previous notebooks (but it helps...). 

Simply run the next few cells by pressing `shift+enter`.

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
font = {'family' : 'normal',
        'size'   : 15}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys
sys.path.insert(0,os.path.join("..", "..", "dependencies"))
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd



To maintain continuity in the series of tutorials, we we use the PEST-dataset prepared in the "obs and weigths" tutorial. Run the next cell to copy fthe necessary files across. Note that if you will need to run the previous notebooks in the correct order beforehand.

Specify the path to the PEST dataset template folder. Recall that we will prepare our PEST dataset files in this folder, keeping them separate from the original model files. Then copy across pre-prepared model and PEST files:

In [ ]:
# specify the temporary working folder
t_d = os.path.join('freyberg6_template')
if os.path.exists(t_d):
    shutil.rmtree(t_d)

org_t_d = os.path.join("..","part2_8_opt","freyberg6_template")
if not os.path.exists(org_t_d):
    raise Exception("you need to run the '/part2_8_opt/freyberg_opt_1.ipynb' notebook")

shutil.copytree(org_t_d,t_d)

In [ ]:
pst_path = os.path.join(t_d, 'freyberg_mf6.pst')

### Inspect the PEST Dataset

OK. We can now get started.

Load the PEST control file as a `Pst` object. We are going to use the PEST control file that was created in the "pstfrom pest setup" tutorial. This control file has observations with weights equal to the inverse of measurement noise (**not** weighted for visibility!).

In [ ]:
pst = pyemu.Pst(pst_path)

### Run PESTPP-MOU

`PESTPP-MOU` implements a constrained multiple and single objective "global" optimization using evolutionary algorithms.  Additional terminology alert!


 - "individual": an optimization problem candidate solution. So just a decision variable vector - one value for each decision variable
 - "population":  well, a collection of individuals
 - "generation": a complete cycle of the evolutionary algorithm (think "iteration"), which involves generating a new, "child" population by combining parents, evaluting the children's fitness (running the population thru the model), and the (natural) selection, where the "best" individuals from the parent and child population are keep.  No doubt "best" is where things get complicated...
 - "generator":  the algorithmic process to generate a child population.  Differential evolution is the default in PESTPP-MOU but there are others
 - "selector": the algorithmic process to pick the best individuals in the population to move to the next generation.  For single objective formulations, this is trivial. For multiobjective formulations, selection is also complex.


Well, there you have it - you are now ready for PESTPP-MOU! but wait, how big should the population be?  How many generations should I use?  Great questions!  Generally it is said that the population should be about twice as large as the number of decision variables.  As for generations, lots (and this is the problem!).  Like 50, 100, or more are not uncommon...




In [ ]:
par = pst.parameter_data
par.loc[:,"partrans"] = "fixed"

dvpar = par.loc[par.pargp=="decvars",:]

par.loc[dvpar.parnme,"partrans"] = "none"
par.loc[dvpar.parnme,"parubnd"] = 1.0

dvpop = pyemu.ParameterEnsemble.from_uniform_draw(pst,num_reals=160)
dvpop.to_csv(os.path.join(t_d,"initial_dvpop.csv"))
pst.pestpp_options["mou_dv_population_file"] = 'initial_dvpop.csv'
par.loc[:,"partrans"] = "none"
par.loc[dvpar.parnme,"parubnd"] = 6.0

In [ ]:
pst.pestpp_options["mou_objectives"] = ["oname:cum_otype:lst_usecol:sfr_totim:4383.5","oname:cum_otype:lst_usecol:wel_totim:4383.5"]

pst.prior_information = pst.null_prior
obs = pst.observation_data
obs.loc[obs.apply(lambda x: x.weight > 0 and "wel" in x.obsnme,axis=1),"weight"] = 0.0
obs.loc[pst.pestpp_options["mou_objectives"],'weight'] = 1.0
obs.loc[pst.pestpp_options["mou_objectives"],'obgnme'] = "less_than_obj"

pst.pestpp_options["mou_population_size"] = 160 #twice the number of decision variables
pst.pestpp_options["mou_save_population_every"] = 1
pst.control_data.noptmax = 0
pst.write(pst_path,version=2)                       

In [ ]:
pyemu.os_utils.run("pestpp-mou freyberg_mf6.pst",cwd=t_d)

In [ ]:
pst.control_data.noptmax = 10
pst.write(pst_path,version=2)

# Attention!

You must specify the number which is adequate for ***your*** machine! Make sure to assign an appropriate value for the following `num_workers` variable:

In [ ]:
num_workers = 15# psutil.cpu_count(logical=False) # update according to your available resources!

Then specify the folder in which the PEST manager will run and record outcomes. It should be different from the `t_d` folder. 

In [ ]:
m_d = os.path.join('master_mou_1')

The following cell deploys the PEST agents and manager and then starts the run using `pestpp-mou`. Run it by pressing `shift+enter`.

If you wish to see the outputs in real-time, switch over to the terminal window (the one which you used to launch the `jupyter notebook`). There you should see `pestpp-mou`'s progress. 

If you open the tutorial folder, you should also see a bunch of new folders there named `worker_0`, `worker_1`, etc. These are the agent folders. `pyemu` will remove them when PEST finishes running.

This run should take a while to complete (depending on the number of workers and the speed of your machine). If you get an error, make sure that your firewall or antivirus software is not blocking `pestpp-mou` from communicating with the agents (this is a common problem!).

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-mou","freyberg_mf6.pst",num_workers=num_workers,worker_root=".",
                           master_dir=m_d)

### Processing PESTPP-MOU

Ok, lets see what's in the that master dir:

In [ ]:
[f for f in os.listdir(m_d) if f.startswith("freyberg_mf6")]

Holy crap thats a lot of files!  A brief description of these files:

 - "freyberg_mf6.XX.dv_pop.csv": the decision variable population at the end of generation XX
 - "freyberg_mf6.XX.obs_pop.csv": the corresponding observation values for the population at the end of generation XX
 - "freyberg_mf6.XX.archive.dv_pop.csv": the archive population at the end of generation XX; the archive only contains "quality" individuals
 - "freyberg_mf6.XX.archive.obs_pop.csv": the corresponding observation values for the archive population at the end generation XX
 - "freyberg_mf6.pareto.summary.csv": a summary of the pareto dominance and feasibility (think "fitness") of each individual across all generations
 - "freyberg_mf6.pareto.acrhive.summary.csv": a summary of the pareto dominance and feasibility (think "fitness") of each individual in the archive across all generations.
 
   
Let's inspect that archive summary file:

In [ ]:
df = pd.read_csv(os.path.join(m_d,"freyberg_mf6.pareto.archive.summary.csv"))
df

In [ ]:
df = df.loc[df.apply(lambda x: x.nsga2_front==1 and x.is_feasible==1 and x.generation == df.generation.max(),axis=1),:]


In [ ]:
fig,axes = plt.subplots(2,2,figsize=(10,10))
objs = pst.pestpp_options["mou_objectives"]
onames = [o.split("usecol:")[1].split('_')[0] for o in objs]
axes[0,0].hist(df.loc[:,objs[0]],alpha=0.5,facecolor="0.5")
axes[0,0].set_title(onames[0])
axes[1,1].set_title(onames[1])

axes[1,1].hist(df.loc[:,objs[1]],alpha=0.5,facecolor="0.5")
axes[1,0].scatter(df.loc[:,objs[0]],df.loc[:,objs[1]],marker=".",c="0.5")
axes[0,1].scatter(df.loc[:,objs[1]],df.loc[:,objs[0]],marker=".",c="0.5")
axes[1,0].set_title("{0} vs {1}".format(onames[0],onames[1]))
axes[0,1].set_title("{0} vs {1}".format(onames[1],onames[0]))
for ax in [axes[0,0],axes[1,1]]:
    ax.set_yticks([])
    ax.set_xlabel("$L^3$")
for ax in [axes[1,0],axes[0,1]]:
    ax.set_xlabel("$L^3$")
    ax.set_ylabel("$L^3$")
    

plt.tight_layout()